In [ ]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# vta: Mini-Challenge Gradient Descent

![image.png](attachment:image.png)

Das Ziel dieser Aufgabe besteht darin, dass Sie ein grundlegendes Verständnis für numerische Näherungsverfahren in höheren Dimensionen erlangen, insbesondere für den Gradient Descent und dessen praktische Anwendung. Hierfür sollen Sie ein Jupyter Notebook erstellen und das MNIST Dataset laden und erkunden. Anschließend sollen Sie ein neuronales Netzwerk erstellen und trainieren, um die Bilder korrekt zu klassifizieren. Es dürfen nur die angegebenen Python packages verwendet werden.

Ziel dieser Aufgabe ist nicht nur, Ihre mathematischen Kenntnisse unter Beweis zu stellen, sondern auch die entsprechende Kommunikation und Präsentation Ihrer Ergebnisse. Ihre Abgaben sollen also nicht nur mathematisch korrekt, sondern auch leicht verständlich und reproduzierbar sein. Genauere Angaben zu den Erwartungen an die Abgabe finden Sie in den Auswertungskriterien. Dokumentieren Sie ihren Arbeitsfortschritt und Erkenntnisgewinn in Form eines Lerntagebuchs, um Lernfortschritte, Schwierigkeiten und Erkenntnisse festzuhalten.
Die folgenden Aufgabenstellungen präzisieren die einzelnen Bearbeitungsschritte und geben die Struktur des Notebooks vor.

## Activation Functions
The activation function is a critical component of neural networks that helps introduce non-linearity to the model. It plays a crucial role in determining the output of a neural network for a given input. With the advent of deep learning, researchers and practitioners have explored a wide range of activation functions to improve the performance of neural networks.

In this chapter, we will provide an overview of the most commonly used activation functions and compare their strengths and weaknesses. We will delve into the different use cases for each activation function and how they impact the training of neural networks.


### Relu

**Advantages of ReLU**:
1. Non-linearity: ReLU introduces non-linearity, allowing the model to learn complex patterns.
$$
f(x) = max(0, x)
$$

2. Computational efficiency: ReLU is computationally efficient due to its simple form.

3. Sparse activation: ReLU promotes sparsity, reducing model complexity.
Why? $f(x) = 0$ for $x < 0$, encouraging fewer active neurons.

**Disadvantages of ReLU**:
1. Dying ReLU: Neurons can "die" if inputs are consistently negative.
Why? $f(x) = 0$ for $x < 0$, leading to no gradient updates.

2. Non-differentiable at 0: ReLU is not differentiable at $x=0$.
Why? Discontinuity at $x=0$.

3. Non-zero centered: ReLU outputs are not zero-centered, causing optimization issues.
Why? Biased gradients can hinder learning.

In [ ]:
def relu(x):
    return np.maximum(0, x)

def relu_grad(x):
    return x > 0

### Sigmoid

**Advantages of Sigmoid**:
1. Smooth, differentiable: Sigmoid is a smooth, differentiable function.
$$
f(x) = \frac{1}{1 + e^{-x}}
$$

2. Range: Sigmoid maps input to $(0, 1)$, providing normalized outputs.
Why? Useful for probabilistic interpretations and binary classification.

3. Non-linearity: Sigmoid introduces non-linearity, allowing complex pattern learning.

**Disadvantages of Sigmoid**:
1. Vanishing gradient: Sigmoid suffers from vanishing gradient problem.
Why? Gradients can be small ($f'(x) \approx 0$) for large $|x|$, slowing learning.

2. Non-zero centered: Sigmoid outputs are not zero-centered, causing optimization issues.
Why? Biased gradients can hinder learning.

3. Computational complexity: Sigmoid is computationally more complex than ReLU.
Why? Exponential calculation in the function.

In [ ]:
def sigmoid(x):
    x = np.clip(x, -500, 500) # clip x to prevent overflow
    return 1 / (1 + np.exp(-x))

def sigmoid_grad(x):
    return sigmoid(x) * (1 - sigmoid(x))

### Softmax

The softmax function is used as the activation function in the output layer of neural network models that predict a multinomial probability distribution. Softmax is used as the activation function for multi-class classification problems where class membership is required on more than two class labels.

**Advantages of Softmax activation function**:
1. Probabilistic interpretation: Softmax produces a probability distribution over the output classes. The sum of the probabilities is 1.

$$
\sigma(z)_i = \frac{e^{z_i}}{\sum_{j=1}^K e^{z_j}}
$$

Where $z_i$ is the input to the Softmax function and $K$ is the number of classes.

2. Differentiable: Softmax is differentiable, which is essential for gradient-based optimization methods like gradient descent.

3. Suitable for multi-class classification: Softmax is well-suited for multi-class classification problems as it produces a probability distribution over all classes.

**Disadvantages of Softmax activation function**:
1. Inappropriate for binary classification: While Softmax can be used for binary classification, it is computationally inefficient compared to using a sigmoid activation function.

2. Susceptible to vanishing gradients: Softmax may result in vanishing gradients when used with certain loss functions (e.g., mean squared error) in deep networks, slowing down learning.

3. Numerical instability: Softmax can lead to numerical instability due to exponentiation of large values. This can be mitigated using the log-sum-exp trick:
$$ 
\sigma(z)_i = \frac{e^{z_i - c}}{\sum_{j=1}^K e^{z_j - c}}
$$
Where $c = \max_i z_i$.

Source: 
- https://machinelearningmastery.com/softmax-activation-function-with-python/
- https://medium.com/intuitionmath/how-to-implement-the-softmax-derivative-independently-from-any-loss-function-ae6d44363a9d

In [ ]:
def softmax(x):
    x = x - np.max(x, axis=1, keepdims=True) # prevent overflow
    x = np.exp(x)
    x = x / np.sum(x, axis=1, keepdims=True)
    return x

def softmax_grad(x):
    return softmax(x) * (1 - softmax(x))

## Loss Functions

The loss function is a key component of neural network training that plays a crucial role in optimizing the model's performance. It is used to measure the difference between the predicted and actual values of the output, and the objective of training is to minimize this difference.

### Mean Squared Error

The Mean Squared Error (MSE) loss function measures the average squared difference between the true values and the predicted values. It is commonly used in regression tasks to quantify the discrepancy between the model's predictions and the ground truth.

$$
L(y, \hat{y}) = \frac{1}{N}(y - \hat{y})^2 
$$


**Advantages of MSE loss**:
1. Continuity and differentiability: MSE is a continuous and differentiable function.

2. Simple to compute: MSE is easy to compute and differentiate.

3. Intuitive interpretation: MSE measures the average squared error between predictions and true values.

4. Commonly used: MSE is a widely-used loss function for regression tasks.

**Disadvantages of MSE loss**:
1. Suboptimal for classification: MSE is more suited for regression tasks.
Why? It doesn't directly optimize for class probabilities or decision boundaries.

2. Sensitivity to outliers: MSE is sensitive to outliers, skewing predictions.
Why? Squaring errors exaggerates the impact of extreme values.

3. Inconsistent gradient magnitude: MSE gradients depend on the error magnitude.
Why? Large errors lead to larger gradients, potentially causing overshooting.

4. Non-probabilistic interpretation: MSE doesn't provide probability calibration.
Why? It doesn't focus on optimizing probabilities, which are useful for thresholding and classification tasks.

In [ ]:
def mse(y_pred, y_true):
    batch_size = y_pred.shape[0]
    loss = np.sum((y_pred - y_true)**2) / batch_size
    return loss

def mse_grad(y_pred, y_true):
    batch_size = y_pred.shape[0]
    grad = 2 * (y_pred - y_true) / batch_size
    return grad

### Binary Cross-Entropy
**Advantages of BCE loss**:
1. Probabilistic interpretation: BCE measures the error between true binary labels and predicted probabilities, optimizing for class probabilities.
$$
L(y, \hat{y}) = -[y \log(\hat{y}) + (1 - y) \log(1 - \hat{y})]
$$

2. Suitable for classification: BCE is designed for binary classification tasks, directly optimizing decision boundaries.

3. Robust to outliers: BCE is less sensitive to outliers compared to MSE, as it doesn't square errors.

4. Stable gradients: BCE provides stable and informative gradient signals for classification tasks, facilitating learning.

**Disadvantages of BCE loss**:
1. Not suitable for multi-class problems: BCE is designed for binary classification and requires modification (e.g., categorical cross-entropy) for multi-class problems.

2. Requires probability input: BCE loss expects predicted probabilities as input, requiring an activation function (e.g., sigmoid) to convert raw model outputs.

3. Logarithm computation: BCE involves logarithm computations, which can be computationally more expensive than simple arithmetic operations (e.g., in MSE).

The gradient:

$$
\begin{aligned}
\frac{\partial L(y)}{\partial y} & =\frac{\partial(-t \log (y)-(1-t) \log (1-y))}{\partial y}=\frac{\partial(-t \log (y))}{\partial y}+\frac{\partial(-(1-t) \log (1-y))}{\partial y} \\
& =-\frac{t}{y}+\frac{1-t}{1-y}=\frac{y-t}{y(1-y)}
\end{aligned}
$$

Sources:
- https://towardsdatascience.com/understanding-binary-cross-entropy-log-loss-a-visual-explanation-a3ac6025181a
- https://towardsdatascience.com/where-did-the-binary-cross-entropy-loss-function-come-from-ac3de349a715
- https://stats.stackexchange.com/questions/219241/gradient-for-logistic-loss-function
- https://peterroelants.github.io/posts/cross-entropy-logistic/

In [ ]:
def binary_cross_entropy(y_pred, y_true):
    eps = 1e-15
    y_pred = np.clip(y_pred, eps, 1-eps) # clip y_pred to prevent log(0) or log(1)
    return -(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred)).mean()

def binary_cross_entropy_grad(y_pred, y_true):
    eps = 1e-15
    y_pred = np.clip(y_pred, eps, 1-eps) # clip y_pred to prevent division by 0
    return (y_pred - y_true) / (y_pred * (1 - y_pred)) # add eps to prevent division by 0

### Focal Loss
$$
FL(p_t)=-\alpha(1-p_t)^\gamma \cdot log(p_t)
$$

Focal Loss is an extension of Cross-Entropy loss, primarily designed to address class imbalance issues in classification tasks. It was introduced by Lin et al. in the paper "Focal Loss for Dense Object Detection."

**Advantages of Focal Loss**:

1. Handles class imbalance: Focal Loss is designed to handle class imbalance by down-weighting the contribution of easy-to-classify examples and focusing on hard-to-classify examples.
$$L(y, \hat{y}) = -(1 - \hat{y})^\gamma y \log(\hat{y}) - \hat{y}^\gamma (1 - y) \log(1 - \hat{y})$$
Where $\gamma$ is the focusing parameter.

1. Adjustable focusing parameter: The focusing parameter $\gamma$ controls the degree of down-weighting for easy examples, allowing users to fine-tune the balance between classes.

2. Compatible with multi-class tasks: Focal Loss can be easily extended to multi-class tasks by adapting the categorical cross-entropy loss.

3. Improved performance: Focal Loss can improve the performance of classification models, especially when class imbalance is present, by helping the model focus on difficult examples.

**Disadvantages of Focal Loss**:

1. Complexity: Focal Loss is more complex compared to traditional cross-entropy loss due to the additional focusing parameter and the computation of the modulating factor.

2. Hyperparameter tuning: The focusing parameter $\gamma$ may require hyperparameter tuning to find the best value for a specific task, adding complexity to the training process.

3. Limited benefits in balanced datasets: The main advantage of Focal Loss is its ability to handle class imbalance. In balanced datasets, it might not provide significant benefits over traditional cross-entropy loss.

In summary, Focal Loss is advantageous when dealing with class imbalance, as it helps the model focus on hard-to-classify examples. However, it introduces additional complexity and requires hyperparameter tuning for the focusing parameter, which might not be beneficial for balanced datasets.

In [ ]:
alpha, gamma = .9, 2

def focal_loss(y_pred, y_true):
    y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7)
    loss = -alpha * (1 - y_pred) ** gamma * y_true * np.log(y_pred) - (1 - y_true) * alpha * y_pred ** gamma * np.log(1 - y_pred)
    return loss

def focal_loss_grad(y_pred, y_true):
    y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7)
    grad = alpha * (1 - y_pred) ** (gamma - 1) * (gamma * y_true * y_pred - (gamma - 1) * y_true * (1 - y_pred) + y_pred * (1 - y_true) * gamma)
    return grad
    
ALPHA = 0.7
GAMMA = 3.0

def binary_focal_loss(y_true, y_pred):
    p = np.clip(y_pred, 1e-7, 1 - 1e-7)
    loss_pos = -ALPHA * np.power(1 - p, GAMMA) * np.log(p) * y_true
    loss_neg = -(1 - ALPHA) * np.power(p, GAMMA) * np.log(1 - p) * (1 - y_true)
    return np.sum(loss_pos + loss_neg)

def binary_focal_loss_gradient(y_true, y_pred):
    p = np.clip(y_pred, 1e-7, 1 - 1e-7)
    term1 = (1 - p)**GAMMA
    term2 = GAMMA * p * (1 - p)**(GAMMA - 1)
    grad_pos = -ALPHA * (term1 * (1 - y_true) - term2 * y_true * np.log(p))
    grad_neg = (1 - ALPHA) * (term1 * y_true - term2 * (1 - y_true) * np.log(1 - p))
    return grad_pos + grad_neg



### Cross-Entropy
$$
 L =-\frac{1}{N}\left(\sum_{i=1}^N \mathbf{y}_{\mathbf{i}} \cdot \log \left(\hat{\mathbf{y}}_{\mathbf{i}}\right)\right)
$$

Cross-entropy is the default loss function to use for multi-class classification problems. 

Cross entropy computes a score that summarizes the average difference between the actual and predicted probability distributions for all classes of the problem. The score is minimized and a perfect cross entropy value is 0.

The ground truth $y$ gives all probability to the first value, and the other values are zero, so we can ignore them, and just use the matching term from the estimate $\hat{\mathbf{y}}$. For example:

$$
\begin{aligned}
& L =-(1 \times \log (0.1)+0 \times \log (0.5)+\ldots) \\
& L =-\log (0.1) \approx 2.303
\end{aligned}
$$

The main feature of this loss function is that only the probabilities of the correct classes are rewarded/punished. The cross-entropy loss function is often averaged over the batch size, so that the loss function is independent of the batch size.

**Advantages of cross-entropy loss function**:
1. Probabilistic interpretation: Cross-entropy loss measures the difference between two probability distributions, making it suitable for classification tasks where the output is a probability distribution.

2. Faster convergence: Cross-entropy loss generally converges faster than other loss functions (e.g., mean squared error) when used with logarithmic activation functions like sigmoid or softmax, as it avoids the vanishing gradient problem.

3. Differentiable: Cross-entropy loss is differentiable, which is crucial for gradient-based optimization methods like gradient descent.

$$
H(y, \hat{y}) = -\sum_{i=1}^N y_i \log(\hat{y}_i)
$$

**Disadvantages of cross-entropy loss function**:
1. Inapplicable to non-probabilistic tasks: Cross-entropy loss is not suitable for regression tasks, where the output is a continuous value rather than a probability distribution.

2. Numerical instability: Cross-entropy loss involves logarithmic operations, which can lead to numerical instability when predicted probabilities are very close to 0 or 1. This issue can be mitigated by clipping the predicted probabilities to a small range (e.g., [1e-8, 1-1e-8]).

3. Requires normalized outputs: Cross-entropy loss assumes that the model's output is a probability distribution. This requires the use of appropriate activation functions, such as softmax for multi-class classification or sigmoid for binary classification, in the output layer.

When using the cross-entropy loss function, the gradient of the loss function is given by:

$$
\frac{\partial L}{\partial z_{j}} = \hat{\mathbf{y}}_i-\mathbf{y}_i
$$

This partial derivative take advantage of the fact that the Softmax activation function is a normalized exponential function. Therefore, the gradient of the cross-entropy loss function is simplified to the difference between the predicted and the actual class probabilities.

Calculating the gradient of the cross-entropy loss function is nicely illustrated in the following image:

![image.png](attachment:image.png)

Sources: 
- https://davidbieber.com/snippets/2020-12-12-derivative-of-softmax-and-the-softmax-cross-entropy-loss/
- https://levelup.gitconnected.com/killer-combo-softmax-and-cross-entropy-5907442f60ba
- https://machinelearningmastery.com/cross-entropy-for-machine-learning/

In [ ]:
def cross_entropy(y_pred, y_true):
    eps = 1e-15
    y_pred = np.clip(y_pred, eps, 1-eps) # clip y_pred to prevent log(0) or log(1)
    loss = -(y_true * np.log(y_pred)).sum(axis=1).mean()
    return loss

def cross_entropy_grad(y_pred, y_true):
    eps = 1e-15
    y_pred = np.clip(y_pred, eps, 1-eps) # clip y_pred to prevent division by 0
    grad = y_pred - y_true
    return grad

### Kullback–Leibler divergence
Kullback Leibler Divergence, or KL Divergence for short, is a measure of how one probability distribution differs from a baseline distribution.

A KL divergence loss of 0 indicates that the distributions are identical. In practice, the behavior of KL divergence is very similar to cross entropy. It calculates how much information is lost when the predicted probability distribution is used to approximate the desired target probability distribution.

**Advantages of KL divergence loss function**:
1. Probabilistic interpretation: KL divergence measures the difference between two probability distributions, making it suitable for tasks where the output is a probability distribution.

2. Asymmetric: KL divergence is asymmetric, making it sensitive to the ordering of the true and predicted distributions. This can be advantageous when the direction of the divergence matters.

$$
D_{KL}(P || Q) = \sum_{i} P(i) \log \frac{P(i)}{Q(i)}
$$

3. Differentiable: KL divergence is differentiable, which is important for gradient-based optimization methods like gradient descent.

**Disadvantages of KL divergence loss function**:
1. Non-negative: KL divergence is non-negative, which may not be ideal for certain optimization algorithms that rely on negative values to indicate convergence.

2. Inapplicable to non-probabilistic tasks: KL divergence is not suitable for regression tasks, where the output is a continuous value rather than a probability distribution.

3. Not symmetric: KL divergence is not symmetric, meaning that $D_{KL}(P || Q) \neq D_{KL}(Q || P)$. In some applications, a symmetric measure of divergence might be more appropriate.

4. Requires normalized outputs: KL divergence assumes that the model's output is a probability distribution. This requires the use of appropriate activation functions, such as softmax for multi-class classification or sigmoid for binary classification, in the output layer.

5. Numerical instability: KL divergence involves logarithmic operations, which can lead to numerical instability when predicted probabilities are very close to 0. This issue can be mitigated by adding a small constant to the predicted probabilities or using a modified version of the KL divergence that avoids division by zero.

## Aufgabe 1
> Laden Sie das MNIST-Dataset mithilfe des torchvision-Pakets (Verwenden Sie das torchvision Paket für diese Aufgabe) und verwenden Sie matplotlib, um sich einen Überblick über die Daten zu verschaffen. Beschreiben Sie das grundlegenden Eigenschaften des Datensets, z.B. wie viele und welche Daten es enthält.

This code performs several data preprocessing steps for the MNIST dataset. 

First, it defines a set of transformations to be applied to the data using the `transforms.Compose` function. The `ToTensor()` function converts the input data from a numpy array to a PyTorch tensor, and the `Normalize()` function normalizes the input tensor with a mean of 0.5 and a standard deviation of 0.5.

Then, it loads the MNIST dataset using the `datasets.MNIST` function and applies the defined transformations to the data. It separates the training and testing data and labels into separate arrays `X_train`, `y_train`, `X_test`, and `y_test`.

Next, it reshapes the data arrays `X_train` and `X_test` from 2D arrays of shape [N, 28, 28] to 1D arrays of shape [N, 784]. This step flattens each image into a 1D vector of 784 values.

Finally, it normalizes the data arrays `X_train` and `X_test` to be between 0 and 1 by dividing each pixel value by 255.0, which is the maximum pixel value.

The final line of code prints the shape of the data and label arrays.

In [ ]:
# Define transformations to be applied to the data
transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize((0.5,), (0.5,))
])

# Load the MNIST dataset
train_set = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_set = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Extract the data and labels from the datasets
X_train, y_train = train_set.data.numpy(), train_set.targets.numpy()
X_test, y_test = test_set.data.numpy(), test_set.targets.numpy()

# Reshape the data to be of size [N x 784]
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

# Normalize the data to be between 0 and 1
X_train = X_train / 255.0
X_test = X_test / 255.0

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
classes = np.unique(y_train)

# Plot the images
fig, axes = plt.subplots(2, 5, figsize=(10, 5))
for i, ax in enumerate(axes.flat):
    ax.imshow(X_train[y_train == classes[i]][0].reshape(28, 28), cmap='gray')
    ax.set_title(f"Class: {classes[i]}")
    ax.axis('off')

plt.show()

In [ ]:
print('Number of training examples: ', X_train.shape[0])
print('Number of testing examples: ', X_test.shape[0])
print('Each image is of size: ', X_train.shape[1])
print('There are {} classes: {}'.format(len(classes), classes))
print('The data is of type: ', X_train.dtype)
print('The labels are of type: ', y_train.dtype)
print('The range of the pixel values is [{}, {}]'.format(np.min(X_train), np.max(X_train)))

In [ ]:
for i in range(len(classes)):
    print(f"Class {classes[i]}: {np.sum(y_train == classes[i])} train examples, {np.sum(y_test == classes[i])} test examples")

## Aufgabe 2

> Erstellen Sie eine Klasse für ein lineares Layer mit beliebig vielen Knoten. Implementieren Sie die Methoden forward, backward und update mithilfe von numpy. Schreiben sie geeignete Unittests, um die Funktionsweise der Funktion zu prüfen.

### Linear Layer

This class represents a linear layer in a neural network. The `__init__` function initializes the layer's parameters, including the input size, output size, activation function, and its derivative. It also initializes the weights and biases randomly with small values.

The `forward` function computes the output of the layer by taking the dot product of the input with the layer's weights and adding the bias. If the activation function is not `None`, it applies it to the linear output and returns the result.

The `backward` function computes the gradient of the loss with respect to the layer's weights, biases, and inputs. If an activation function exists, it computes the gradient of the loss with respect to the output before applying the activation function. Then it computes the gradient of the loss with respect to the input by taking the dot product of the gradient of the loss with respect to the output and the transpose of the layer's weights.

The `update` function updates the weights and biases using gradient descent by subtracting the product of the learning rate and the gradients from the current values.

Overall, this class implements a fully connected layer with an optional activation function that can be used in a neural network.

In [ ]:
class LinearLayer:
    def __init__(self, input_size, output_size, activation_fn, activation_fn_grad):
        self.x = None
        self.bias_grad = None
        self.weights_grad = None

        self.input_size = input_size
        self.output_size = output_size
        self.activation_fn = activation_fn
        self.activation_fn_grad = activation_fn_grad
        self.weights = np.random.randn(input_size, output_size) * 0.01
        self.bias = np.zeros((1, output_size))

    def forward(self, x):
        self.x = x
        linear_output = np.dot(x, self.weights) + self.bias
        
        if self.activation_fn is None:
            return linear_output
        
        return self.activation_fn(linear_output)

    def backward(self, grad_output, hidden_output=None):
        if self.activation_fn_grad is not None:
            if hidden_output is not None:
                grad_output = self.activation_fn_grad(hidden_output) * grad_output
            else:
                grad_output = self.activation_fn_grad(grad_output) * grad_output
        
        self.weights_grad = np.dot(self.x.T, grad_output)
        self.bias_grad = np.sum(grad_output, axis=0, keepdims=True)
        return np.dot(grad_output, self.weights.T)

    def update(self, lr):
        self.weights -= lr * self.weights_grad
        self.bias -= lr * self.bias_grad

### Unit Testing

This code defines a unit test for the `LinearLayer` class using the `unittest` module. The test class inherits from `unittest.TestCase`, and it contains several test functions:

- `setUp` function initializes the parameters of the `LinearLayer` class for testing purposes.
- `test_forward` function tests if the `forward` function of the `LinearLayer` class returns an output with the correct shape.
- `test_backward` function tests if the `backward` function of the `LinearLayer` class returns an input gradient with the correct shape.
- `test_update` function tests if the `update` function of the `LinearLayer` class updates the weights and biases correctly.
- `test_shapes` function tests the shapes of the weight and bias gradients returned by the `backward` function.

The `unittest.main` function runs the test cases and returns the results. By running these test cases, we can ensure that the `LinearLayer` class and its functions are working correctly.

In [ ]:
import unittest

class TestLinearLayer(unittest.TestCase):
    def setUp(self):
        self.input_size = 10
        self.output_size = 5
        self.linear_layer = LinearLayer(self.input_size, self.output_size, None, None)

    def test_forward(self):
        x = np.random.randn(1, self.input_size)
        output = self.linear_layer.forward(x)
        self.assertEqual(output.shape, (1, self.output_size))

    def test_backward(self):
        x = np.random.randn(1, self.input_size)
        output = self.linear_layer.forward(x)
        grad_output = np.random.randn(1, self.output_size)
        grad_input = self.linear_layer.backward(grad_output)
        self.assertEqual(grad_input.shape, (1, self.input_size))

    def test_update(self):
        self.linear_layer.weights = np.array([[1, 2], [3, 4]], dtype=np.float32)
        self.linear_layer.bias = np.array([[1, 2]], dtype=np.float32)
        self.linear_layer.weights_grad = np.array([[1, 2], [3, 4]], dtype=np.float32)
        self.linear_layer.bias_grad = np.array([[1, 2]], dtype=np.float32)

        self.linear_layer.update(0.1)
        self.assertTrue(np.allclose(self.linear_layer.weights, np.array([[0.9, 1.8], [2.7, 3.6]])))
        self.assertTrue(np.allclose(self.linear_layer.bias, np.array([[0.9, 1.8]])))

    def test_shapes(self):
        x = np.random.randn(1, self.input_size)
        output = self.linear_layer.forward(x)
        grad_output = np.random.randn(1, self.output_size)
        grad_input = self.linear_layer.backward(grad_output)

        self.assertEqual(self.linear_layer.weights_grad.shape, (self.input_size, self.output_size))
        self.assertEqual(self.linear_layer.bias_grad.shape, (1, self.output_size))

unittest.main(argv=[''], verbosity=2, exit=False)

## Aufgabe 3

> Erstellen Sie ein neuronales Netzwerk in numpy mit einem Hidden Linear Layer und einem Output Knoten. Trainieren Sie das Netzwerk darauf, die Ziffer 4 korrekt zu identifizieren (d.h. der Output soll 1 für diese Ziffer und 0 für alle anderen Ziffern sein). Trainieren Sie das Netzwerk auf den Trainingsdaten und evaluieren Sie es anhand von Testdaten. Verwenden Sie eine geeignete Loss-Funktion sowie Accuracy-Funktion und geben Sie deren mathematische Definition an. Begründen Sie Ihre Wahl mit einer Abwägung der Vor- und Nachteile. Diskutieren Sie kurz weitere Optionen für Loss und Accuracy.

### Loss function
**Binary Cross Entropy (BCE)** is a suitable loss function for the MNIST binary classification task of classifying if a digit is a 4 or not, mainly because it is specifically designed for binary classification problems. Mathematically, BCE calculates the negative log likelihood of the correct class, penalizing the model heavily when its prediction is far from the ground truth. This characteristic makes BCE sensitive to the difference between predicted probabilities and actual labels, thereby promoting accurate classification.

In comparison, Focal loss is designed to address class imbalance issues by down-weighting the loss contribution from easily classified examples. While this property makes Focal loss an optimal choice for the given task with a 9:91 class distribution, the inability to implement it necessitated the use of BCE. Nonetheless, BCE still provides a reasonable performance in such scenarios. To address the imbalance issue, remixing the training and test set can be considered. 

Mean Squared Error (MSE) loss, on the other hand, is suboptimal for classification tasks, as it focuses on the squared difference between predictions and ground truth. This leads to less effective gradient updates, particularly in the presence of saturating activation functions like sigmoid. Moreover, MSE is more suitable for regression tasks rather than binary classification, and its use may result in subpar performance for the given task.

In summary, Binary Cross Entropy is a suitable choice for the MNIST binary classification task, especially when Focal loss is not implementable. While BCE may not directly address class imbalance, its mathematical properties make surely a solid choice.


### Accuracy function

The F1 score is an appropriate accuracy metric for the MNIST binary classification task of identifying 4s due to the imbalanced nature of the dataset, where 4s constitute approximately 9% and non-4s make up 91%. The F1 score combines precision (the proportion of true positive 4s among all predicted 4s) and recall (the proportion of true positive 4s among all actual 4s) through their harmonic mean. This balances false positives and false negatives, ensuring that both are adequately addressed.

In contrast, metrics like naive-accuracy can be misleading, as a classifier that predicts all non-4s will achieve high accuracy (91%) despite failing to identify any 4s. While other metrics like precision, recall, or the area under the ROC curve (AUC-ROC) can provide useful insights, the F1 score remains a more comprehensive and balanced choice for this specific classification task. The F1 score is calculated like this:

$$
\begin{aligned}
& \text { Precision }=\frac{T P}{T P+F P} \\
& \text { Recall }=\frac{T P}{T P+F N} \\
& F 1 \text { score }=2 * \frac{\text { Precision } * \text { Recall }}{\text { Precision }+ \text { Recall }}
\end{aligned}
$$

In [ ]:
def f1_score(y_pred, y_true):
    y_true = np.array(y_true).flatten()
    y_pred = np.array(y_pred).flatten()
    assert y_true.shape == y_pred.shape, "y_true and y_pred must have the same shape"

    y_true = y_true.flatten()
    y_pred = y_pred.flatten()

    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))

    if tp + fp == 0 or tp + fn == 0:
        return 0

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    f1 = 2 * (precision * recall) / (precision + recall)
    return f1

def plot_confusion_matrix(cm, class_labels=None, normalize=False, title='Confusion Matrix', cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        fmt = '.2f'
    else:
        fmt = 'd'

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)

    if class_labels is None:
        class_labels = ['0', '1']

    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels=class_labels, yticklabels=class_labels,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")

    fig.tight_layout()
    plt.show()

def confusion_matrix(y_true, y_pred, plot=True, plot_title='Confusion Matrix'):
    y_true = np.array(y_true).flatten()
    y_pred = np.array(y_pred).flatten()
    assert y_true.shape == y_pred.shape, "y_true and y_pred must have the same shape"

    y_true = y_true.flatten()
    y_pred = y_pred.flatten()

    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fn = np.sum((y_true == 1) & (y_pred == 0))

    cm = np.array([[tn, fp], [fn, tp]])

    if plot:
        plot_confusion_matrix(cm, title=plot_title)

    return cm

### Tracking network development
In order to track the development of the training process, we define a class that stores the training and validation loss and accuracy for each epoch. We also define a function that plots the training and validation loss and accuracy for each epoch. This allows us to see how the training process develops over time.

The `NetworkDevelopment` class is responsible for storing the training and validation loss and accuracy for each epoch during the training process of a neural network. It has the following methods:

- `__init__(self, total_epochs)`: Initializes the class with the total number of epochs the network will be trained for. It initializes empty lists to store the training and validation losses and accuracies for each epoch.
- `add_epoch(self, epoch_number, loss, acc_train, acc_test)`: Adds a new development step to the network development by appending the loss, accuracy for training data, and accuracy for validation data of the current epoch to their respective lists. This method returns a string containing the epoch number, loss, and accuracies for training and validation data.
- `plot(self)`: Plots the training and validation loss and accuracy for each epoch using matplotlib. It creates two subplots, one for loss over epochs and another for accuracy over epochs. The method returns the plot.
- `summary(self)`: Prints the final loss and accuracy of the training and validation data, and the average improvements per epoch. It calls the `plot()` method to plot the loss and accuracy data, and then prints the average accuracy change per epoch for both the training and validation data, and the average loss change per epoch.

In [ ]:
class NetworkDevelopment:
    def __init__(self, total_epochs):
        self.total_epochs = total_epochs
        self.losses = []
        self.accuracies_train = []
        self.accuracies_test = []

    def add_epoch(self, epoch_number, loss, acc_train, acc_test):
        self.losses.append(loss)
        self.accuracies_train.append(acc_train)
        self.accuracies_test.append(acc_test)

        return f'Epoch {epoch_number}/{self.total_epochs} - loss: {loss:.4f} - acc_train: {acc_train:.4f} - acc_test: {acc_test:.4f}'

    def plot(self):
        epochs = np.arange(1, self.total_epochs + 1)

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        ax1.plot(epochs, self.losses)
        ax1.set_title('Loss over Epochs')
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Loss')
        ax2.plot(epochs, self.accuracies_train, label='Training')
        ax2.plot(epochs, self.accuracies_test, label='Test')
        ax2.set_title('Accuracy over Epochs')
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('Accuracy')
        ax2.legend()
        plt.show()

    def summary(self):
        self.plot()

        print(f'avg acc change / epoch (Training set): {np.mean(np.diff(self.accuracies_train)):.4f}')
        print(f'avg acc change / epoch (Test set): {np.mean(np.diff(self.accuracies_test)):.4f}')
        print(f'avg loss change / epoch: {np.mean(np.diff(self.losses)):.4f}')

### Batch Training

Training a neural network on large datasets can be computationally expensive and time-consuming. One way to mitigate this challenge is to use batch training, where instead of updating the model's weights after every single data point, the model's parameters are updated after processing a fixed number of data points, known as a batch.

The function `get_batches(x, y, batch_size)` creates batches of images and labels from the training set. The number of images in a batch is defined by the batch size. The batch size is a hyperparameter that can be tuned to improve the training process. The batch size is a trade-off between the number of images used for training and the number of training steps per epoch. A larger batch size results in fewer training steps per epoch, but the training process is less accurate. A smaller batch size results in more training steps per epoch, but the training process is more accurate.

- `x`: A numpy array containing the input data.
- `y`: A numpy array containing the target data.
- `batch_size`: An integer representing the size of each batch.

The function first calculates the number of batches needed to cover the entire dataset by dividing the length of the input data by the batch size. The function then shuffles the input and target data using a random permutation of indices to ensure that the data is not ordered in any particular way. 

The function then uses a generator to yield batches of input and target data. It does this by iterating over the range of indices from 0 to the number of batches times the batch size, with a step size of batch size. For each iteration, it slices the input and target data arrays to extract a batch of size batch_size, and then yields this batch as a tuple of (x_batch, y_batch).

In [ ]:
def get_batches(x, y, batch_size):
    n_batches = len(x) // batch_size
    
    # shuffle data before creating batches
    idx = np.random.permutation(len(x))
    x = x[idx]
    y = y[idx]

    for i in range(0, n_batches * batch_size, batch_size):
        x_batch = x[i:i+batch_size]
        y_batch = y[i:i+batch_size]
        yield x_batch, y_batch

### Single-Layer Network

This code defines a single-layer neural network class `SingleLayerNetwork` that inherits from the `LinearLayer` class. The `SingleLayerNetwork` class has several methods:

- `__init__` initializes the network's parameters, including the input size, hidden layer size, output size, loss function, and its derivative. It also initializes the hidden and output layers using `LinearLayer` with the appropriate activation functions.
- `forward` computes the forward pass of the network by propagating the input through the hidden and output layers using their `forward` functions.
- `backward` computes the backward pass of the network by propagating the gradient of the loss with respect to the output through the output and hidden layers using their `backward` functions.
- `update` updates the weights and biases of the hidden and output layers using their `update` functions and the learning rate.
- `train` trains the network on the given training data `X_train` and labels `y_train` for a specified number of epochs using mini-batch gradient descent. It computes the accuracy and loss on the training and test data at each epoch and outputs the results if `output` is `True`.
- `predict` makes predictions on the given input data `X` by calling the `forward` function and returning the predictions.
- `evaluate` evaluates the accuracy of the network's predictions on the given input data `X` and labels `y`.
- `summary` prints a summary of the network's development using the `NetworkDevelopment` class.

Overall, this class defines a simple single-layer neural network and its training and evaluation methods.

In [ ]:
class SingleLayerNetwork:
    def __init__(self, input_size, hidden_size, output_size, loss_fn, loss_fn_grad, acc_fn):
        self.loss_fn = loss_fn
        self.loss_fn_grad = loss_fn_grad
        self.acc_fn = acc_fn

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.hidden_layer = LinearLayer(input_size, hidden_size, relu, relu_grad)
        self.output_layer = LinearLayer(hidden_size, output_size, sigmoid, sigmoid_grad)

    def forward(self, x):
        self.x = x
        self.h = self.hidden_layer.forward(x)
        self.y_pred = self.output_layer.forward(self.h)
        return self.y_pred

    def backward(self, y_true):
        y_true = y_true.reshape(-1, 1)
        self.y_pred = self.y_pred.reshape(-1, 1)

        grad_output = self.loss_fn_grad(self.y_pred, y_true)

        grad_output = self.output_layer.backward(grad_output)
        grad_output = self.hidden_layer.backward(grad_output)

        return grad_output

    def update(self, lr):
        self.hidden_layer.update(lr)
        self.output_layer.update(lr)

    def train(self, X_train, y_train, X_test, y_test, lr, epochs, batch_size, output=True):
        self.dev = NetworkDevelopment(total_epochs=epochs)
        
        for epoch in range(epochs):
            loss_list = []
            for x_batch, y_batch in get_batches(X_train, y_train, batch_size):
                y_pred = self.forward(x_batch)

                loss = self.loss_fn(y_pred, y_batch)
                loss_list.append(loss)

                self.backward(y_batch)
                self.update(lr)

            acc_train = self.evaluate(X_train, y_train)
            acc_test = self.evaluate(X_test, y_test)
            avg_loss = np.mean(loss_list)

            epoch_str = self.dev.add_epoch(epoch+1, avg_loss, acc_train, acc_test)
            if output:
                print(epoch_str)

    def predict(self, X):
        return (self.forward(X) > .5).astype(int).reshape(-1, 1)

    def evaluate(self, X, y):
        y_pred = self.predict(X).flatten()
        # return np.mean(y_pred == y)
        return self.acc_fn(y_pred, y)

    def summary(self):
        self.dev.summary()

### Training

In order to train the model for the task at hand, we need to convert the $y$ data into a binary format. Each label representing a $4$ is converted into an $1$. Every non-$4$ label is converted to a $0$.

In [ ]:
y_train_4_binary = (y_train == 4).astype(int)
y_test_4_binary = (y_test == 4).astype(int)

We now train the Single-Layer Network on the train and test data with binary encoded labels.

In [ ]:
slp = SingleLayerNetwork(input_size=784, hidden_size=512, output_size=1, loss_fn=binary_cross_entropy, loss_fn_grad=binary_cross_entropy_grad, acc_fn=f1_score)
slp.train(X_train, y_train_4_binary, X_test , y_test_4_binary, lr=0.00001, epochs=15, batch_size=256)
slp.summary()

### Modifying datasets to counter bias

Given the nature of the default test set, there is a natural bias against 4s, since they are only making up around 9% of train and test set. If the model would be trained on this default set, it would develop a strong bias against 4s and reach a 90%> accuracy even though it is not capable of actually identifying a 4.

To make sure that our model can identify 4s and non-4s equally well we created some subsets which will allow to train and test the model in a less biased way. To accomplish this, a mixed train and test set with equal distribution of 4s and non-4s is created (mix). All non-4 digits have equal representation in the set. 

The model will be trained on this mixed set and then evaluated against two other sets: 4s only and non-4s. The goal is to reach a high accuracy in all 3 test sets, since this means that the model can correctly identify 4s and non-4s and doesn't have strong bias towards either one. Afterwards the model will evaluate the default test set.

Only after having a high accuracy in all 4 test sets(mix, 4s, non-4s and default), can we conclude the models success.


The code is preparing three datasets, one containing only images of the number 4 (y=1), one with only images of other digits (y=0), and one with an equal representation of both. 


It first converts the original labels to binary values where 1 represents the number 4, and 0 represents any other digit. 

Then, it separates the images of each digit in the training and testing sets. It calculates the number of samples to be selected from each class, which will be the same as the number of images of 4 in each subset, and selects those from all digits except for 4. This results in two subsets containing an equal number of 4 and non-4 images. 

The code shuffles the two subsets separately and converts the labels to binary. 

Afterward, the code selects only the 4 images from the original training and testing sets, shuffles them, and also converts their labels to binary. 

Finally, the code creates a new mixed dataset by taking an equal number of samples from both the 4s-only and non-4s-only subsets, shuffles it, and converts its labels to binary. The output of the total number of 4s and non-4s in the training set is printed.

In [ ]:
# Separate the digits
digits = [X_train[y_train == i] for i in range(10)]
digits_test = [X_test[y_test == i] for i in range(10)]

# Calculate the number of samples for each not 4 class
samples_per_class = len(digits[4]) // 9
samples_per_class_test = len(digits_test[4]) // 9

# Prepare the not 4s subset with equal representation of every other digit
X_train_not_4 = np.concatenate([digits[i][:samples_per_class] for i in range(10) if i != 4])
y_train_not_4 = np.concatenate([np.full(samples_per_class, i) for i in range(10) if i != 4])

X_test_not_4 = np.concatenate([digits_test[i][:samples_per_class_test] for i in range(10) if i != 4])
y_test_not_4 = np.concatenate([np.full(samples_per_class_test, i) for i in range(10) if i != 4])

# Shuffle the not 4s subset
indices_not_4 = np.random.permutation(len(X_train_not_4))
X_train_not_4 = X_train_not_4[indices_not_4]
y_train_not_4 = y_train_not_4[indices_not_4]

indices_not_4_test = np.random.permutation(len(X_test_not_4))
X_test_not_4 = X_test_not_4[indices_not_4_test]
y_test_not_4 = y_test_not_4[indices_not_4_test]

y_train_not_4_binary = (y_train_not_4 == 4).astype(int)
y_test_not_4_binary = (y_test_not_4 == 4).astype(int)


# Shuffle the 4s subset
indices_4 = np.random.permutation(len(digits[4]))
X_train_4_only = digits[4][indices_4]
y_train_4_only = np.full(len(digits[4]), 4)


indices_4_test = np.random.permutation(len(digits_test[4]))
X_test_4_only = digits_test[4][indices_4_test]
y_test_4_only = np.full(len(digits_test[4]), 4)

y_train_4_only_binary = (y_train_4_only == 4).astype(int)
y_test_4_only_binary = (y_test_4_only == 4).astype(int)


# Create the mixed dataset with a 50:50 split
X_train_mix = np.concatenate((X_train_4_only[:len(X_train_not_4)], X_train_not_4))
y_train_mix = np.concatenate((y_train_4_only[:len(y_train_not_4)], y_train_not_4))

X_test_mix = np.concatenate((X_test_4_only[:len(X_test_not_4)], X_test_not_4))
y_test_mix = np.concatenate((y_test_4_only[:len(y_test_not_4)], y_test_not_4))


# Shuffle the mixed dataset
indices_mix = np.random.permutation(len(X_train_mix))
X_train_mix = X_train_mix[indices_mix]
y_train_mix = y_train_mix[indices_mix]

indices_mix_test = np.random.permutation(len(X_test_mix))
X_test_mix = X_test_mix[indices_mix_test]
y_test_mix = y_test_mix[indices_mix_test]

# Convert the labels to binary
y_train_mix_binary = (y_train_mix == 4).astype(int)
y_test_mix_binary = (y_test_mix == 4).astype(int)

# Print the count of 4s and not 4s in the train set
print(f"4s: {np.sum(y_train_mix_binary == 1)} - Not 4s: {np.sum(y_train_mix_binary == 0)}")

### Training with modified datasets

In [ ]:
slpm = SingleLayerNetwork(input_size=784, hidden_size=512, output_size=1, loss_fn=binary_cross_entropy, loss_fn_grad=binary_cross_entropy_grad, acc_fn=f1_score)
slpm.train(X_train_mix, y_train_mix_binary, X_test_mix, y_test_mix_binary, lr=0.00001, epochs=30, batch_size=128)
slpm.summary()

### Testing




In [ ]:
confusion_matrix(y_test_4_binary, slp.predict(X_test), plot_title="SLP")
confusion_matrix(y_test_4_binary, slpm.predict(X_test), plot_title="SLPM")

In [ ]:
f1_score(y_test_4_binary, slp.predict(X_test)), f1_score(y_test_4_binary, slpm.predict(X_test))

In [ ]:
acc_4_only = slp.evaluate(X_test_4_only, y_test_4_only_binary)
acc_not_4 = slp.evaluate(X_test_not_4, y_test_not_4_binary)
print(f"Accuracy on 4s: {acc_4_only} - Accuracy on not 4s: {acc_not_4}")

normal_acc = slp.evaluate(X_test, y_test_4_binary)
print(f"Accuracy on normal test set: {normal_acc}")

In [ ]:
acc_4_only = slpm.evaluate(X_test_4_only, y_test_4_only_binary)
acc_not_4 = slpm.evaluate(X_test_not_4, y_test_not_4_binary)
print(f"Accuracy on 4s: {acc_4_only} - Accuracy on not 4s: {acc_not_4}")

normal_acc = slpm.evaluate(X_test, y_test_4_binary)
print(f"Accuracy on normal test set: {normal_acc}")

## Aufgabe 4

> Trainieren Sie das Netzwerk mit verschiedenen Lernraten und Größen des Hidden Layers. Verfolgen Sie während des Trainings die Entwicklung der Loss- und Accuracy-Funktionen auf Trainings- und Testdatensätzen und entscheiden Sie, welche Wahl von Lernrate und Hidden Layer-Größe die besten Ergebnisse in geringster Zeit liefert.

In order to find the right hyperparameters, we define a function that trains the network with different learning rates, epochs, hidden layer sizes and batch sizes. The result for each combination is then stored and analyzed.

This brute force approach is not very efficient, but it allows us to find the best hyperparameters for the network.

> **Note:** The brute force process takes a long time to complete. Therefore, we have commented out the code that runs the brute force process. The best result is described further below.

In [ ]:
def get_param_combinations(epochs, batch_sizes, learning_rates, hidden_sizes):
    return [(e, b, l, h) for e in epochs for b in batch_sizes for l in learning_rates for h in hidden_sizes]

epochs = [10, 20, 30]
batch_sizes = [2**i for i in range(5, 11)]
learning_rates = [10**-i for i in range(1, 8)]
hidden_sizes = [64, 128, 256, 512]

def test_brute_force_single_layer(param_combos, X_train, y_train, X_test, y_test):
    results = []
    best_acc = 0
    best_param_combo = None

    for i in range(len(param_combos)):
        try:
            e, b, l, h = param_combos[i]
            network = SingleLayerNetwork(input_size=784, hidden_size=h, output_size=1, 
                                        loss_function=binary_cross_entropy, loss_function_grad=binary_cross_entropy_grad,
                                        acc_fn=f1_score)

            network.train(X_train, y_train, X_test, y_test, lr=l, epochs=e, batch_size=b, output=False)
            acc = network.evaluate(X_test, y_test)

            results.append((e, b, l, h, accuracy))
            if acc > best_acc:
                best_acc = acc
                best_param_combo = param_combos[i]

            print(f"combo {i+1}/{len(param_combos)}: - Epochs: {e} - Batch size: {b} - Learning rate: {l} - Hidden size: {h} - Accuracy: {acc:.4f}")
        except Exception as e:
            print(f"Error: {e}")
            continue

    return best_acc, best_param_combo


sl_combos = get_param_combinations(epochs, batch_sizes, learning_rates, hidden_sizes)
print(f"Testing {len(sl_combos)} parameter combinations")

In [ ]:
test_brute_force_single_layer(sl_combos, X_train, y_test_4_binary, X_test, y_test_4_binary)

The best result was achieved with the following hyperparameters:

## Aufgabe 5

> Erweitern Sie das Netzwerk auf 3 Hidden Layer mit gleicher Größe und 10 Outputs. Das Ziel ist die korrekte Klassifizierung aller Ziffern. Verwenden Sie eine geeignete Loss-Funktion sowie Accuracy-Funktion und geben Sie deren mathematische Definition an. Begründen Sie Ihre Wahl und diskutieren Sie kurz weitere Möglichkeiten. Variieren Sie die Lernrate und die Größe der Hidden Layer und wählen Sie das beste Ergebnis aus.

### Loss function

**Cross Entropy loss** is a preferred choice for the MNIST classification task due to its ability to handle probabilistic outputs effectively. This is especially useful when dealing with multi-class classification problems like MNIST, which involves 10 different digit classes. Mathematically, Cross Entropy loss quantifies the difference between the predicted probability distribution and the true distribution, incentivizing the model to increase the probability of correct predictions.

Comparatively, Mean Squared Error (MSE) is more suited for regression problems and lacks the same discriminative power as Cross Entropy when handling classification tasks. MSE doesn't exploit the probabilistic nature of class predictions, which can lead to slower convergence and less accurate results.

While Kullback-Leibler (KL) Divergence also measures the dissimilarity between two probability distributions, it is asymmetric and can be computationally expensive. Cross Entropy, on the other hand, is a symmetric and more computationally efficient alternative for classification tasks. 

In conclusion, Cross Entropy loss is an optimal choice for the MNIST classification problem due to its capability to handle probabilistic outputs, faster convergence, and computational efficiency compared to alternatives like MSE and KL Divergence.

### Accurcay function
The standard accuracy metric is the most suitable choice for the MNIST classification task, as it effectively captures the proportion of correctly classified examples. Mathematically, accuracy is calculated by dividing the number of correct predictions by the total number of predictions, yielding a value between 0 and 1. This simple metric is easy to interpret and understand, making it ideal for evaluating the performance of a classifier on a balanced dataset like MNIST, where each digit class is equally represented.

$$
\text{Accuracy} = \frac{\text{Number of Correct Predictions}}{\text{Total Number of Predictions}} = \frac{\sum_{i=1}^{n} I(y_i = \hat{y}_i)}{n}
$$

Alternatives like precision, recall, and F1-score are generally more informative for imbalanced datasets, where certain classes are underrepresented. However, since the MNIST dataset for all classes (see exercise 1) is balanced, these metrics provide little additional value compared to standard accuracy. Moreover, the Area Under the Receiver Operating Characteristic curve (AUROC) and Area Under the Precision-Recall curve (AUPRC) are more relevant for binary classification problems or when a probabilistic output is required, which is not the case for MNIST.

In conclusion, the standard accuracy metric is the most appropriate choice for the MNIST classification task due to its simplicity, interpretability, and effectiveness in capturing the classifier's performance on balanced datasets. Alternative metrics like precision, recall, F1-score, AUROC, and AUPRC provide limited additional insight for this specific problem.

### One-Hot Encoding

One-hot encoding is a way to represent categorical data in a numerical format. In the case of MNIST, the target variable is the digit that each image represents, which can take on values from 0 to 9. One-hot encoding converts each digit into a binary vector of length 10, where each element of the vector represents a possible digit value. For example, the digit 3 would be represented as [0, 0, 0, 1, 0, 0, 0, 0, 0, 0], where the fourth element is a 1 and the other elements are 0.

In [ ]:
def one_hot_encode(y):
    y_one_hot = np.zeros((len(y), 10))
    y_one_hot[np.arange(len(y)), y] = 1
    return y_one_hot

y_train_one_hot = one_hot_encode(y_train)
y_test_one_hot = one_hot_encode(y_test)

### Multi-Layer Network

The `MultiLayerNetwork` class represents a neural network with multiple layers, and is used for training and evaluating the model on a given dataset. The class implements the functions necessary for forward and backward propagation in the network. 

The `__init__` function initializes the network with the given input size, hidden size, output size, loss function, and loss function gradient. It also initializes the various layers of the network, including three hidden layers and an output layer.

The `forward` function performs forward propagation on a given input `x`, computing the predicted output of the network. The `backward` function performs backward propagation to compute the gradients of the network's weights and biases with respect to the loss, given the true output `y_true`. The `update` function updates the weights and biases of the network based on the gradients computed in the backward pass.

The `train` function trains the network on the given training data `X_train` and `y_train`, using the given learning rate, number of epochs, and batch size. It also computes the training and validation accuracy and loss for each epoch using the `evaluate` function, and stores these values in a `NetworkDevelopment` object. 

The `predict` function takes a set of input data `X` and returns the predicted classes of each input based on the current weights and biases of the network. The `evaluate` function takes a set of input data `X` and corresponding true output `y`, and computes the accuracy of the predicted classes relative to the true classes.

Finally, the `summary` function prints a summary of the network's training and validation accuracy and loss for each epoch, as well as the average improvements per epoch, using the `NetworkDevelopment` object.

In [ ]:
class MultiLayerNetwork:
    def __init__(self, input_size, hidden_size, output_size, loss_function, loss_function_grad):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.loss_function = loss_function
        self.loss_function_grad = loss_function_grad

        self.hidden_layer1 = LinearLayer(input_size, hidden_size, relu, relu_grad)
        self.hidden_layer2 = LinearLayer(hidden_size, hidden_size, relu, relu_grad)
        self.hidden_layer3 = LinearLayer(hidden_size, hidden_size, relu, relu_grad)
        self.output_layer = LinearLayer(hidden_size, output_size, softmax, softmax_grad)

        self.dev = None
        self.hidden_output1 = None
        self.hidden_output2 = None
        self.hidden_output3 = None
        self.predicted_output = None

    def forward(self, x):
        self.hidden_output1 = self.hidden_layer1.forward(x)
        self.hidden_output2 = self.hidden_layer2.forward(self.hidden_output1)
        self.hidden_output3 = self.hidden_layer3.forward(self.hidden_output2)
        self.predicted_output = self.output_layer.forward(self.hidden_output3)
        return self.predicted_output

    def backward(self, y_true):
        gradient_output = self.loss_function_grad(self.predicted_output, y_true)

        gradient_output = self.output_layer.backward(gradient_output, self.predicted_output)
        gradient_output = self.hidden_layer3.backward(gradient_output, self.hidden_output3)
        gradient_output = self.hidden_layer2.backward(gradient_output, self.hidden_output2)
        gradient_output = self.hidden_layer1.backward(gradient_output, self.hidden_output1)

        return gradient_output

    def update(self, learning_rate):
        self.hidden_layer1.update(learning_rate)
        self.hidden_layer2.update(learning_rate)
        self.hidden_layer3.update(learning_rate)
        self.output_layer.update(learning_rate)

    def train(self, X_train, y_train, X_test, y_test, learning_rate, epochs, batch_size, output=True):
        self.dev = NetworkDevelopment(total_epochs=epochs)

        for epoch in range(epochs):
            loss_list = []
            for X_batch, y_batch in get_batches(X_train, y_train, batch_size):
                predicted_output = self.forward(X_batch)

                loss = self.loss_function(predicted_output, y_batch)
                loss_list.append(loss)

                self.backward(y_batch)
                self.update(learning_rate)

            avg_loss = np.mean(loss_list)
            acc_train = self.evaluate(X_train, y_train)
            acc_test = self.evaluate(X_test, y_test)

            epoch_str = self.dev.add_epoch(epoch+1, avg_loss, acc_train, acc_test)
            if output:
                print(epoch_str)

    def predict(self, X):
        predicted_output = self.forward(X)
        return np.argmax(predicted_output, axis=1)

    def evaluate(self, X, y):
        predicted_classes = self.predict(X)
        true_classes = np.argmax(y, axis=1)
        return np.mean(predicted_classes == true_classes)

    def summary(self):
        self.dev.summary()

### Training

In [ ]:
mln = MultiLayerNetwork(input_size=784, hidden_size=256, output_size=10, loss_function=cross_entropy, loss_function_grad=cross_entropy_grad)
mln.train(X_train, y_train_one_hot, X_test, y_test_one_hot, learning_rate=0.01, epochs=10, batch_size=64)
mln.summary()

### Testing

In [ ]:
# evaluate the network on the test set
print(f"Accuracy: {mln.evaluate(X_test, y_test_one_hot):.4f}")

### Finding the right hyperparameters

In [ ]:
epochs = [10*i for i in range(1, 5)]
batch_sizes = [2**i for i in range(5, 11)]
learning_rates = [10**-i for i in range(1, 6)]
hidden_sizes = [64, 128, 256, 512]
combinations = get_param_combinations(epochs, batch_sizes, learning_rates, hidden_sizes)

def test_brute_force_multi_layer(param_combos, X_train, y_train, X_test, y_test):
    results = []
    best_acc = 0
    best_param_combo = None

    for i in range(len(param_combos)):
        try:
            e, b, l, h = param_combos[i]
            net = MultiLayerNetwork(input_size=784, hidden_size=h, output_size=10, loss_function=cross_entropy, loss_function_grad=cross_entropy_grad)

            net.train(X_train, y_train_one_hot, X_test, y_test_one_hot, learning_rate=l, epochs=e, batch_size=b, output=False)
            acc = net.evaluate(X_test, y_test_one_hot)

            results.append((e, b, l, h, accuracy))
            if acc > best_acc:
                best_acc = acc
                best_param_combo = param_combos[i]

            print(f"combo {i+1}/{len(param_combos)}: - Epochs: {e} - Batch size: {b} - Learning rate: {l} - Hidden size: {h} - Accuracy: {acc:.4f}")
        except Exception as e:
            print(f"Error: {e}")
            continue

    return best_acc, best_param_combo

ml_combos = get_param_combinations(epochs, batch_sizes, learning_rates, hidden_sizes)
print(f"Testing {len(ml_combos)} parameter combinations")

In [ ]:
test_brute_force_multi_layer(ml_combos, X_train, y_train, X_test, y_test)

## Lerntagebuch

### KW13

### KW14

- Multiplikation Linear Layer vertauscht? input x weights or weights x input

### KW15

### KW16

#### 17.04.2023
SingleLayerModel:

Loss-Function Decision: We found that there are a bunch of different loss functions that are used in the context of neural networks. We wanted to take a deeper look into how each of those functions would perform in the case of our Single-Layer-Network and Multi-Layer-Network.

Focal: 

Hinge:

Final: Binary-Cross-Entropy.

Explanation: 

---
Geeignete Loss- und Accuracy-Funktionen
- Die Wahl wurde begründet und mit anderen mögliche Funktionen verglichen?
- Die mathematische Definition der Loss-Funktion und Accuracy-Funktion ist korrekt angege- ben (gerendert in Latex)?
- Die Entwicklung der Loss- und Accuracy-Funktionen wurden auf Trainings- und Testdaten- sätzen korrekt verfolgt und leicht nachvollziehbar dargestellt?
- Die Wahl von Lernrate und Hidden Layer-Größe wurde nachvollziehbar entschieden und begründet.


---

For the Muli-class classification problem:

**Loss Function Choice**: 
Kullback: Complicated integration due to requirements
MSE: Rather weak, due to the focus on regression

Cross-Entropy is ideal for Tasks 4 and 5 because it handles classification problems with probability predictions well. Other loss functions, such as Mean Squared Error, are less suitable because they are more focused on regression and have no direct connection to the classification of probabilities. Therefore, Cross-Entropy is the best choice.

**Accuracy Functions**:
Accuracy: Correct / False. Basic and sufficient.
F1-Score: Perhaps better for exercise 3 to counteract imbalance in the dataset (90% accuracy due to few 4s != good) but not necessary for exercise 5.

Confusion Matrix: Also probably more helpful for exercise 3. 

**Activation Functions**:

ReLU: The best practice in the current age. Very efficient and doesn't share the problems that sigmoid has (vanishing gradients leading to slower learning and/or convergence)

Leaky ReLU: ReLU but with a better focus on making sure that dead neurons (neurons that are updated, so that they can never be activated again) are taken care of.

Parametric ReLU:
PReLU is another variation of ReLU that generalizes Leaky ReLU by making the negative slope a learnable parameter. Higher risk of overfitting.

Sigmoid: Is just bad now, no reason to ever consider it except for an educational purpose. The main problem lies in vanishing gradients leading to slower learning and/or convergence: The vanishing gradient problem is a difficulty encountered during the training of deep neural networks using gradient-based optimization methods, such as gradient descent and its variants. This issue arises when the gradients of the loss function with respect to the network's parameters become very small as they propagate through the layers, leading to slow or stalled learning. This can happen at any stage in the training process, which makes it more difficult to counteract it properly.

---

Geeignete Loss- und Accuracy-Funktionen wurden verwendet.
- Die Wahl wurde begründet und mit anderen möglichen Funktionen verglichen.
- Die mathematische Definition der Loss-Funktion und Accuracy-Funktion ist korrekt angegegeben (gerendert in Latex).
- Es wurden verschiedene Lernraten und Größen der Hidden Layer sinnvoll ausprobiert.
- Die Entwicklung der Loss- und Accuracy-Funktionen auf Trainings- und Testdatensätzen wurde korrekt verfolgt und leicht nachvollziehbar dargestellt?
- Die Wahl der Hyperparameter wurde nachvollziehbar entschieden und begründet?

Form:

Environment angeben (Python 3.10)
- Code comments
- Executive Summary



#### 18.04.2023

Fragen an Stefan:
- Verständnis Anzahl Layers
- Aufgabe 3: Biased Training? Interpretation von unseren Resultaten?
- Keine Gradienten Berechnung im LinearLayer (Punkt 5)
- Geeignete Loss- und Accuracy-Funktionen wurden verwendet.
- Lerntagebuch auch im Notebook? Summary on top?
- Time import bei Aufgabe 4 erlaubt? (Zeitmessung oder Epochen gemeint)

#### 19.04.2023

* Based on the talk with Stefan it appears as if our accuracy decision for the single layer network is not ideal. So we tried out some accuracy functions, namely: F1-Score, Precision

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=88bc6171-47da-4cbc-96f1-ee851c7ac9ec' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>